In [158]:
from IPython.display import display, HTML,clear_output
import random as rnd
import math
import time

In [159]:
class SVGCanvas:
    def __init__(self,width,height,size):
        self.width = width
        self.height = height
        self.size = size
        self.html_s =""
    
    def clear(self):
        self.html_s = '<svg width="'+str(self.width)+'" height="'+str(self.height)+'"style="border:1px solid black">'

    def addCircle(self,x_pos,y_pos,color):
        x_pos = x_pos/self.size
        y_pos = y_pos/self.size
        r = 1/(self.size*2)
        str_int_x = str(int((x_pos*self.width)+(r*self.width)))
        str_int_y = str(int((y_pos*self.height)+(r*self.width)))
        str_int_r = str(int(r*self.width))
        self.html_s +='<circle cx="'+str_int_x+'" cy="'+str_int_y+'" r="'+str_int_r+'" fill="'+color+'"/>'

    def addRect(self,x_pos,y_pos,color):
        x_pos = x_pos/self.size
        y_pos = y_pos/self.size
        height = 1/self.size
        width = 1/self.size
        str_int_x = str(int(x_pos*self.width))
        str_int_y = str(int(y_pos*self.height))
        str_int_width = str(int(width*self.width))
        str_int_height = str(int(height*self.height))
        self.html_s +='<rect x="'+str_int_x+'" y="'+str_int_y+'" width="'+str_int_width+'" height="'+str_int_height+'" fill="'+color+'"/>'

    def getCanvas(self):
        self.html_s += "</svg>"
        return HTML(self.html_s)


In [160]:
class Cell:
    def __init__(self,resource):
        self.occupant = None
        self.resource = resource
        
    def update(self):
        self.resource = rnd.random()

In [161]:
class Agent:
    world=None
    
    def __init__(self,x_pos,y_pos):
        self.x_pos=x_pos
        self.y_pos=y_pos
        Agent.world.grid[x_pos][y_pos].occupant = self
        
    def move(self,dx,dy):
        new_x_pos = Agent.world.bounds(self.x_pos+dx)
        new_y_pos = Agent.world.bounds(self.y_pos+dy)
        if Agent.world.grid[new_x_pos][new_y_pos].occupant==None:
            Agent.world.grid[self.x_pos][self.y_pos].occupant=None
            self.x_pos = new_x_pos
            self.y_pos = new_y_pos
            Agent.world.grid[self.x_pos][self.y_pos].occupant = self
        


In [166]:
class World:
    def __init__(self,size,number_of_agents):
        h_size = int(size/2)
        self.size =size
        self.grid=[]
        self.svg = SVGCanvas(301,301,size)
        for y in range(size):
            self.grid.append([])
            for x in range(size):
                dx = h_size-abs(h_size-x)/h_size*0.5
                dy = h_size-abs(h_size-y)/h_size*0.5
                self.grid[y].append(Cell(dx+dy))
        Agent.world = self
        
        self.agents=[]
        for i in range(number_of_agents):
            flag = True
            while flag:
                x_pos = rnd.randint(0,self.size-1)
                y_pos = rnd.randint(0,self.size-1)
                if self.grid[x_pos][y_pos].occupant==None:
                    self.agents.append(Agent(x_pos,y_pos))
                    flag = False
                                       
        
    def bounds(self,pos):
        if pos<0:
            return self.size+pos
        if pos>= self.size:
            return pos-self.size
        return  pos
                                    
    def draw(self):
        self.svg.clear()
        for i in range(self.size):
            for j in range(self.size):
                b = str(255- int(self.grid[i][j].resource*255))
                self.svg.addRect(i,j,'rgb(255,255,'+b+')')
                if self.grid[i][j].occupant !=None:
                    self.svg.addCircle(i,j,"rgb(255,0,0)")
        display(self.svg.getCanvas())
                                 
    def update(self):
        for a in self.agents:
            a.move(rnd.randint(-1,1),rnd.randint(-1,1))
    
    def run(self,run_time): 
        for t in range(run_time):
            clear_output(wait=True)
            self.update()
            self.draw()                            
            time.sleep(0.01)
        
        
        
        

In [167]:
world = World(30,100)


In [ ]:
world.run(100)